## TIC Info from Astroquery

### Module imports

In [1]:
from astroquery.mast import Observations, Catalogs
from astropy.io import fits
from astropy.table import Table
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### Functions

In [2]:
def import_lightcurve(file_path, drop_bad_points=False,
                      ok_flags=[5]):
    """Returns (N by 2) table, columns are (time, flux).

    Flags deemed to be OK are:
    5 - reaction wheel zero crossing, matters for short cadence
    """

    try:
        hdulist = fits.open(file_path)
    except FileNotFoundError:
        print("Import failed: file not found")
        return

    scidata = hdulist[1].data
    table = Table(scidata)['TIME','PDCSAP_FLUX','QUALITY']

    if drop_bad_points:
        bad_points = []
        q_ind = get_quality_indices(table['QUALITY'])
        for j,q in enumerate(q_ind):
            if j+1 not in ok_flags:
                bad_points += q.tolist()

        # bad_points = [i for i in range(len(table)) if table[i][2]>0]
        table.remove_rows(bad_points)

    # Delete rows containing NaN values.
    nan_rows = [ i for i in range(len(table)) if
            math.isnan(table[i][1]) or math.isnan(table[i][0]) ]

    table.remove_rows(nan_rows)

    # Smooth data by deleting overly 'spikey' points.
    spikes = [ i for i in range(1,len(table)-1) if \
            abs(table[i][1] - 0.5*(table[i-1][1]+table[i+1][1])) \
            > 3*abs(table[i+1][1] - table[i-1][1])]

    for i in spikes:
        table[i][1] = 0.5*(table[i-1][1] + table[i+1][1])

    return table


### Beta Pic

In [3]:
tic = 270577175
obsTable = Observations.query_criteria(provenance_name='TESS-SPOC',target_name=tic)
data = Observations.get_product_list(obsTable)

In [4]:
#download_lc = Observations.download_products(data)

In [7]:
ls

Kepler/
MAD figs/
README.md
__pycache__/
all_snr_gt_5.txt
all_snr_gt_5_ok.txt
analysis_tools_cython.c
analysis_tools_cython.cpython-37m-darwin.so*
analysis_tools_cython.cpython-38-darwin.so*
analysis_tools_cython.cpython-39-darwin.so*
analysis_tools_cython.pyx
archive_analyse.sh*
artefact_list.txt
batch_analyse.py*
batch_reanalyse_asym.py*
box_sn_gt5_maybeTransit.awk
build/
data/
dr2.xml
environment.yml
figs/
figs.ipynb
filters_to_final.awk
lightcurve.ipynb
loaders.py
make*
mastDownload/
maxsector.py
notebooks/
setup.py*
single_analysis.py*
single_analysis_xrp.py
tess/
tess2018206045859-s0001-0000000024265755-0120-s_lc.fits
test.py
young-cl.xml


In [6]:
hdulist = fits.open('hlsp_tess-spoc_tess_phot_0000000270577175-s0006_tess_v1_lc.fits')
hdulist[1].data

FileNotFoundError: [Errno 2] No such file or directory: 'hlsp_tess-spoc_tess_phot_0000000270577175-s0006_tess_v1_lc.fits'

In [ ]:
table = import_lightcurve('hlsp_tess-spoc_tess_phot_0000000270577175-s0006_tess_v1_lc.fits')

In [ ]:
plt.plot(table['TIME'],table['PDCSAP_FLUX'])

---

### Testing glob things

In [ ]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#[print(f"target found") for i in glob.glob('tess/2_min_cadence_targets/*.pkl') if i.endswith('.pkl')]

In [ ]:
def lightcurves(path):
    data = pd.read_pickle(path)
    return data[3]

In [ ]:
lightcurves('tess/2_min_cadence_targets/tesslc_100479167.pkl')

In [ ]:
test = []
for i in glob.glob('tess/2_min_cadence_targets/*.pkl'):
    test.append(lightcurves(i))

In [ ]:
values, base = np.histogram(test, bins=20)

In [ ]:
cumulative = np.cumsum(values) / len(test)


In [ ]:
plt.figure(figsize=(13,4))
plt.plot(base[:-1], cumulative, c='blue')
plt.yscale('log')
plt.title('Distribution of magnitiudes for 2 min targets')
plt.xlabel('Magnitude')
plt.ylabel('Fraction')
plt.show()

Saturated stars ~ 6.8 Mag [Sullivan et al 2015](https://arxiv.org/pdf/1506.03845.pdf), [Feinstein et al. 2019](https://ui.adsabs.harvard.edu/abs/2019PASP..131i4502F/abstract)

In [ ]:
lim = 6.8
non_saturated = [i for i in test if i < lim] # less than because magnitudes work the other way

In [ ]:
len(non_saturated)/len(test) * 100

---

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt

In [ ]:
with fits.open('tess2018206045859-s0001-0000000024265755-0120-s_lc.fits','readonly') as hdulist:
    time = hdulist[1].data['TIME']
    flux = hdulist[1].data['SAP_FLUX']

In [ ]:
len(time)

In [ ]:
plt.plot(time,flux)

---

In [ ]:
from astroquery.mast import Catalogs

In [ ]:
catalog_data = Catalogs.query_criteria(catalog="Tic",Tmag = [1,15])

In [ ]:
catalog_data